<a href="https://colab.research.google.com/github/ortizfram/predicting-stock-prices/blob/main/prediciton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install import libraries

# pip install pandas-datareader
import pandas as pd
import numpy as np 
import pandas_datareader as web
import matplotlib.pyplot as plt
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropou, LSTM